In [4]:
import altair as alt
import pandas as pd
from altair import datum

In [2]:
selectCrime2 = pd.read_csv("selectCrime2.csv")

In [5]:
sampleCrime = selectCrime2.groupby('crimeType', group_keys=False).apply(lambda x: x.sample(min(len(x), 200)))
alt.data_transformers.enable('json')

selection = alt.selection_multi(fields=['crimeType'], bind='legend')

chart=alt.Chart(sampleCrime).transform_window(
    index='count()'
).transform_fold(
    ['All','wall', 'lives','building','infrastructure','road','sidewalk','sky','green','transportation']
).transform_joinaggregate(
     min='min(value)',
     max='max(value)',
     groupby=['key']
).transform_calculate(
    minmax_value=(datum.value-datum.min)/(datum.max-datum.min)
).mark_line(size=0.5).encode(
    x=alt.X('key:N',axis=alt.Axis(ticks=False, domain=False,labelAngle=0)),
    y=alt.Y('minmax_value:Q',axis=alt.Axis(ticks=False, domain=False)),
    color=alt.condition(selection,
                        alt.Color('crimeType:N', scale=alt.Scale(scheme='category20b')),
                        alt.value('lightgray')),   
    opacity=alt.condition(selection, alt.value(1), alt.value(0.07)),   
    detail='index:N',
).properties(
    width=1000,
    height=500,
    title="Relationship between street component and crime type"
).configure_view(
    strokeWidth=0
).add_selection(selection).interactive()

chart.encoding.x.title = 'Street Component'
chart.encoding.y.title = 'Normalized porportion'
chart

alt.Chart(...)